# Importing required packages

In [1]:
#imports
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
import requests
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import random 
import os
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
import pyautogui
import urllib
from PIL import Image
from io import BytesIO


# Defining functions

In [2]:
def wait():
    rand = random.randrange(2000000) / 800000
    time.sleep(rand)

In [3]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    This function was develop by user: Greenstick (https://stackoverflow.com/users/2206251/greenstick)
    on https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console
    
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [4]:
def get_info(browser,reactive = True):
    
    #browser.execute_script("window.open('');")
    #browser.switch_to.window(browser.window_handles[1])
    #browser.get(link)
    if reactive:
    
        try:      
            date = browser.find_element_by_xpath("/html/body/div[4]/div[2]/div/article/div[3]/div[2]/a/time").text
        except:
            date = np.nan
        try:
            text = browser.find_element_by_xpath("/html/body/div[4]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span").text

        except:
            text = np.nan

        #try:
         #   politician = browser.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/header/div[2]/div[1]/div[1]/span/a').text
        #except:
         #   politician = np.nan
        #try:
         #   verified = browser.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/header/div[2]/div[1]/div[1]/div/span').text
        #except:
         #   verified = np.nan

        try:
            likes = browser.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/section[2]/div/div/a').text
        except:
            likes = np.nan

        #try:
         #   location = browser.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/header/div[2]/div[2]/div[2]/a').text
       # except:
        #    location = np.nan

        #browser.close()
        #browser.switch_to.window(browser.window_handles[0])
    else:
        try:      
            date = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[2]/a/time').text
        except:
            date = np.nan
        try:
            text = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span').text

        except:
            text = np.nan
        
        try:
            likes = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/section[2]/div/div/a').text
        except:
            likes = np.nan


    return date,text,likes

In [5]:
def scroll(browser, timeout, **kwargs):
    scroll_pause_time = timeout
    
    folder = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\"+ user +"\\"
    pslink_name = folder + user + "_postlinks.txt"
    piclink_name =  folder + user + "_piclinks.txt"
    
    if os.path.isfile(pslink_name) and os.path.isfile(piclink_name):
        with open(pslink_name, 'r') as file:
            post_links = file.read.splitlines()
        with open(piclink_name, 'r') as file:
            pic_links = file.read.splitlines()
        #pic_links = [post[:-1] for post in pic_links]
        #post_links = [post[:-1] for post in post_links]
        
        return post_links, pic_links
        
    else:
    
        post_links = []
        pic_links = []




        # Get scroll height
        last_height = browser.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(scroll_pause_time)

            #get post links 
            try:
                post_element = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[3]/article/div[1]/div')
            except:
                time.sleep(2)
                try:
                    post_element = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[3]/article/div[1]/div')
                except:
                    post_element = browser.find_element_by_tag_name("article")
            links = post_element.find_elements_by_tag_name('a')

            for i, n in enumerate(links):
                href = links[i].get_attribute('href')

                if href not in post_links:
                    post_links.append(href)

            #get picture links
            pic_elements = browser.find_elements_by_class_name('KL4Bh')

            for i, n in enumerate(pic_elements):
                href = pic_elements[i].find_element_by_tag_name("img").get_attribute("src")

                if href not in pic_links:
                    pic_links.append(href)

            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # If heights are the same it will exit the function
                break
            last_height = new_height

            #if the scroll has not ended - test if the last link is before 6 month before August 26th of 2020
            # that means if the post is before Februrary 26th i stop the scroll 
            #open new tab
            browser.execute_script("window.open('');")
            browser.switch_to.window(browser.window_handles[1])
            browser.get(post_links[-1])

            soup = bs(browser.page_source)

            try:
                date = soup.time['datetime']

                b = cutoff_day(date)

            except:
                date = np.nan

            #close the tab
            browser.close()
            #switch back to scrolling tab
            browser.switch_to.window(browser.window_handles[0])
            #test if we are past cutoff day
            if b:
                break
        return post_links, pic_links

In [6]:
def cutoff_day(date):    
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[8:10])
    
    if year < 2020:
        b = True
    elif year == 2020 and month < 2:
        b = True
    elif year == 2020 and month == 2 and day < 26:
        b = True
    else: 
        b = False
    return b

In [7]:
def use_links(post_links,pic_links, **kwargs):
    ##################### use the links ###################################
    folder = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\"+ user +"\\"
    #print("Saving images to:",folder)
    #print("The user is:",user)
    
    #posts = pd.DataFrame(columns= ["post_nr","date","text","politician","verified","likes","location"])
    
    if not os.path.isdir(folder):
        os.makedirs(folder)
        #get information on the posts
    
    #request pictures   
    l = len(pic_links)
    print("Gathering pictures")
    printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

    for i, n in enumerate(pic_links):

        number = random.randrange(70)
        if number == 1:                                  
            #print("taking a random brake")
            time.sleep(60)
        name = folder + "pic_" + str(i) + ".jpeg"
        if not os.path.isfile(name):
            try:
                image = Image.open(BytesIO(requests.get(pic_links[i]).content))                
                image.save(name)
                #urllib.request.urlretrieve(pic_links[i],name)
            except: 
                image = Image.open('C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\backup.jpg')
                image.save(name)
                image.close()
        time.sleep(2)
        printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

    
    #####browser = start_new_browser()
    #l = len(post_links)
    #print("Gathering information")
    #printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
    #for i, n in enumerate(post_links):
        
        #breaks
     #   number = random.randrange(70)
      #  if number == 1:
            
            #print("taking a random brake")
            #browser.close()
            #browser = start_new_browser()
       #     time.sleep(60)
            

        #get data
        #date, text, politician, verified, likes, location = get_info(browser,post_links[i])
            
       # posts.loc[len(posts)] = [str(i),date,text,politician,verified,likes,location]
       # printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

        #wait()   

        
    #posts["posts_link"] = post_links
    #posts["pic_link"] = pic_links
    
    #filename = folder + user +".csv"
    #posts.to_csv(filename)

In [8]:
def all_links(user):
    url = 'https://www.instagram.com/'+user+'/?hl=en'
    # driver.get(url) opens the page
    browser.get(url)
    # This starts the scrolling by passing the driver and a timeout
    post_links, pic_links = scroll(browser, 5)
    print("finished getting links")
    
    # Once scroll returns bs4 parsers the page_source
    #all_soup = bs(browser.page_source, 'html.parser')
    # Them we close the driver as soup_a is storing the page source
    #browser.close()
    
    #get the data
  
    return post_links, pic_links

In [25]:
def start_new_browser():
    browser = webdriver.Chrome('chromedriver')
    browser.get('https://www.instagram.com/?hl=en')
    browser.implicitly_wait(3)
    browser.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/button[1]").click()
    
    username = "kaarevisted"#"borgerjensen"#"gertjensen123" 
    password =  "Håberdetvirker!!"#"Håberdetvirker!"#"Turpas123!" 
    
    browser.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(username)
    wait()
    browser.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(password)
    wait()
    browser.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
    browser.implicitly_wait(5)
    
    wait()
    try:
        browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
    except: 
        pass
    wait()
    try:
        browser.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
    except:
        browser.find_element_by_class_name("aOOlW   HoLwm").click()

    return browser

In [10]:
def click_next_and_save(user, post_link_start, amount_of_posts):
    ''' user is the user to be scraped 
        post link start is the link to the post to start by 
        amount of posts is the amount of posts that we want, including the starting post.
        
        the function needs an active browser 
    '''
    url = 'https://www.instagram.com/'+user+'/?hl=en'
    browser.get(url)
    browser.implicitly_wait(2)
    
    #find all the links of the first page:
    posts = browser.find_elements_by_class_name("v1Nh3")
    for i, p in enumerate(posts):
        link = posts[i].find_elements_by_tag_name('a')
        if link[0].get_attribute('href') == post_link_start:
            #print("starting at link", i+1)
            break
        elif i == len(posts):
            i = 0
            print("Could not find start link on first page. Starting on first post")
    
    #make a folder to save the html to 
    folder = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\"+ user +"\\html\\"
    if not os.path.isdir(folder):
        os.makedirs(folder)

    #make a while loop 
    posts = pd.DataFrame(columns= ["post_nr","date","text","likes"])

    n = 0
    
    printProgressBar(0, amount_of_posts, prefix = 'Progress:', suffix = 'Complete', length = 50)

    while n < amount_of_posts:
        if n == 0:
            #i is the number of the first post found earlier n is the ittertaion in this loop
            browser.find_elements_by_class_name("_9AhH0")[i].click()    
            filename = folder +"pic_" + str(n) +".html"
            browser.implicitly_wait(2)
            html_file= open(filename,"w",encoding='utf-8')
            html_file.write(str(browser.page_source))
            html_file.close()
            
            #get info
            date,text,likes = get_info(browser)
            
            posts.loc[n] = [str(n),date,text,likes]


        else:
            browser.find_element_by_class_name('_65Bje').click()
            filename = folder + "pic_" + str(n) +".html"
            html_file= open(filename,"w",encoding='utf-8')
            browser.implicitly_wait(2)
            html_file.write(str(browser.page_source))
            html_file.close()
            
            #random break
            number = random.randrange(30)
            if number == 1:                                  
            #print("taking a random brake")
                time.sleep(60)
            
            date,text,likes = get_info(browser)
            
            posts.loc[n] = [str(n),date,text,likes]
                
    #take a break

        printProgressBar(n + 1, amount_of_posts, prefix = 'Progress:', suffix = 'Complete', length = 50)
    #update the while loop by looking at the len of the list of link (eg i == len - 1 (because of 0 indexing))
        n += 1
        time.sleep(2)
    
    
    filename = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\" + user +"\\"+user + ".csv"
    posts.to_csv(filename)

# Scraping

Jeg begynder nu scraping. Der foregår i et par tring:

    - Jeg åbner en start instagram browser. Hvis login skal ændres er det i start_new_browser functionen
    - Jeg definere de brugere jeg vil scrabe
    - herefter sætter jeg scraberen igang. 
    - Det tager ca. 0.3 minut (20 sek) at hente 1 post med alt infomation og billede 
    - Forvent derfor alt i mellem 10 min og 90 min for en bruger alt afhængig af antal billeder. 

In [32]:
browser = start_new_browser()

In [31]:
with open("brugere.txt",'r') as file:
    users = file.read().splitlines()
    #users = [post[:-1] for post in users]
len(users)

1

In [33]:
howmany = 0

In [ ]:
#set to true to use links instead of click
warning = False
for user in users:
    if howmany >= 35:
        break
    
    if warning: 
        begin = time.time()

        print("Working on", user)

        with open("posts\\" + user +"\\" + user +"_postlinks.txt",'r') as file:
            links  = file.read().splitlines()

        folder = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\"+ user +"\\html\\"
        if not os.path.isdir(folder):
            os.makedirs(folder)

        #users = [post[:-1] for post in users]
        #startlink = links[0]
        #num = len(links)
        posts = pd.DataFrame(columns= ["post_nr","date","text","likes"])

        printProgressBar(0, len(links), prefix = 'Progress:', suffix = 'Complete', length = 50)

        for i_,link in enumerate(links):
            browser.get(link)
            browser.implicitly_wait(4)

            #get the html

            filename = folder + "pic_" + str(i_) +".html"
            html_file= open(filename,"w",encoding='utf-8')
            html_file.write(str(browser.page_source))
            html_file.close()

            #get the data

            date,text,likes = get_info(browser,reactive = False)

            posts.loc[i_] = [str(i_),date,text,likes]

            printProgressBar(i_ +1, len(links), prefix = 'Progress:', suffix = 'Complete', length = 50)


        filename = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\" + user +"\\"+user + ".csv"
        posts.to_csv(filename)

        time.sleep(2)

        end = time.time()
        print(f"Finished with user: {user}")
        print(f"Total runtime of the program is for user {user} is {(end - begin) /60} min") 

        browser.close()
        time.sleep(130)

        browser = start_new_browser()
        browser.implicitly_wait(10)

    else:

        begin = time.time()

        print("Working on", user)

        with open("posts\\" + user +"\\" + user +"_postlinks.txt",'r') as file:
            links  = file.read().splitlines()
        #users = [post[:-1] for post in users]
        startlink = links[0]
        num = len(links)

        #get the data
        click_next_and_save(user,startlink,num)

        end = time.time()
        print(f"Finished with user: {user}")
        print(f"Total runtime of the program is for user {user} is {(end - begin) /60} min") 

        browser.close()
        time.sleep(130)
        browser = start_new_browser()
        browser.implicitly_wait(10)
        
    howmany += 1


Working on jakobellemann
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: jakobellemann
Total runtime of the program is for user jakobellemann is 9.162133884429931 min


# Hent billeder 
Er kørt - tager omkring 6 timer

In [41]:
with open("brugere.txt",'r') as file:
    users = file.read().splitlines() 
    #users = [post[:-1] for post in users]


In [42]:
for user in users:
    begin = time.time()

    print("Working on", user)
    print("Getting links")
    post_links, pic_links = all_links(user)
    
    folder = "C:\\Users\\augus\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\Posts\\"+ user +"\\"
    #check if folder exists, otherwise make it
    if not os.path.isdir(folder):
        os.makedirs(folder)
    
    filename = folder + user + "_postlinks.txt"
    file = open(filename, 'w')
    for element in post_links:
        file.write(element)
        file.write('\n')

    # closing the file
    file.close()
    
    filename = folder + user + "_piclinks.txt"
    file = open(filename, 'w')
    for element in pic_links:
        file.write(element)
        file.write("\n")

    # closing the file
    file.close()
    
    print("Fisnished saving the links, now moving onto getting the data")
    
    use_links(post_links,pic_links, user=user)
    
    end = time.time()
    print(f"Finished with user: {user}")
    print(f"Total runtime of the program is for user {user} is {(end - begin) /60} min") 
    
    browser.close()
    browser = start_new_browser()

Working on j.h.thulesen.dahl
Getting links
finished getting links
Fisnished saving the links, now moving onto getting the data
Gathering pictures
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: j.h.thulesen.dahl
Total runtime of the program is for user j.h.thulesen.dahl is 0.5415247241655986 min
Working on kristianthulesendahl
Getting links
finished getting links
Fisnished saving the links, now moving onto getting the data
Gathering pictures
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: kristianthulesendahl
Total runtime of the program is for user kristianthulesendahl is 4.5873776316642765 min
Working on mettedencker
Getting links
finished getting links
Fisnished saving the links, now moving onto getting the data
Gathering pictures
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: mettedencker
Total runtime of the program is for user m

Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: michaelaastrupjensen
Total runtime of the program is for user michaelaastrupjensen is 3.216863199075063 min
Working on jan_e_joergensen
Getting links
finished getting links
Fisnished saving the links, now moving onto getting the data
Gathering pictures
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: jan_e_joergensen
Total runtime of the program is for user jan_e_joergensen is 4.729035969575246 min
Working on marcus.knuth
Getting links
finished getting links
Fisnished saving the links, now moving onto getting the data
Gathering pictures
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Finished with user: marcus.knuth
Total runtime of the program is for user marcus.knuth is 3.0870989481608073 min
Working on stenknuth
Getting links
finished getting links
Fisnished saving the links, now moving onto getting the da

'carl_valentin'